# 1. Basic Probability

## 1.1 Introduction

The world is an uncertain place. Making predictions about something as seemingly mundane as tomorrow’s weather, for example,
is actually quite a difficult task. Even with the most advanced computers and models of the modern era, weather forecasters still cannot say with absolute certainty whether it will rain tomorrow. The best they can do is to report their best estimate of the chance that it will rain tomorrow. For example;

- if the forecasters are fairly confident that it will rain tomorrow, they might say that there is a 90% chance of rain. 

You have probably heard statements like this your entire life, but have you ever asked yourself **what exactly it means to say that there is a 90% chance of rain?**


Let us consider an even more basic example: **tossing a coin**. If the coin is fair, then it is just as likely to come up heads as it is to come up tails. In other words, **if we were to repeatedly toss the coin many times, we would expect about about half of the tosses to be heads and and half to be tails**. In this case, we say that the probability of getting a head is 1/2 or 0.5.

Note that when we say the probability of a head is 1/2, we are not claiming that any sequence of coin tosses will consist of exactly 50% heads. If we toss a fair coin ten times, it would not be surprising to observe 6 heads and 4 tails, or even 3 heads and 7 tails. But as we continue to toss the coin over and over again, we expect the long-run frequency of heads to get ever closer to 50%. In general, it is important in statistics to understand the distinction between theoretical and
empirical quantities. Here, **the true (theoretical) probability of a head was 1/2**, but any **realized (empirical) sequence of coin tosses may have more or less than exactly 50% heads**.

In [1]:
import numpy as np
from bokeh.io import show,output_notebook
from bokeh.models import ColumnDataSource, CustomJS, HoverTool
from bokeh.layouts import gridplot, column, row
from bokeh.events import ButtonClick
from bokeh.models.widgets import Button, Slider
from bokeh.plotting import Figure
import random
output_notebook()

Loading BokehJS ...

In [2]:
# initial values to bar plot
coin = ['Head (H)', 'Tail (T)']
counts = [0,0]


# basic data structure 
source = ColumnDataSource(data=dict(x=coin, y=counts))

# add a hover tip to figure
hover = HoverTool(
    tooltips=[
        ( 'Probability', '@y')
    ]
)

# create a generic figure
p = Figure(title="Observed Outcomes", 
           plot_height=300, 
           plot_width=600, 
           x_range=coin, 
           tools=["pan,wheel_zoom,save", hover])


# add bar plot and indicate who is the source
p.vbar(x='x', top='y', width=0.2, source=source)
p.xgrid.grid_line_color = None
p.y_range.start = 0

In [3]:
# callbacks

#slider head
update_slider_head = CustomJS(code="""

    if (cb_obj.title == "Head (H) probability"){
            prob_a = master_slider.value
            slave_slider.value = 1 - prob_a 
    } else{
            prob_a = slave_slider.value
            master_slider.value = 1 - prob_a 
    }
""")

update_flip_coin = CustomJS(args=dict(source=source), code="""
        data = source.data
        cont = 0
        prob = slider.value
        
        if (cb_obj.label == 'Flip the Coin')
            n = 1
        else
            n = 100
        
        for (i = 0; i < n; i++) {
           if  (Math.random() <= prob)
                cont = cont + 1
        }
        
        y = data['y']
        y[0] = cont/n
        y[1] = 1 - y[0]
        
        console.log(source.data.y,cb_obj.label)
        source.change.emit()
        """)

In [4]:
# create the widgets

head_slider = Slider(start=0.0, 
                     end=1.0, 
                     value=0.5, 
                     step=.01, 
                     title="Head (H) probability",
                     callback=update_slider_head)

tail_slider = Slider(start=0.0, 
                     end=1.0, 
                     value=0.5, 
                     step=.01,
                     title="Tail (T) probability",
                     callback=update_slider_head)

button_one_flip = Button(label="Flip the Coin", 
                         button_type="primary",
                         callback=update_flip_coin)

button_100_flip = Button(label="Flip 100 times", 
                         button_type="primary",
                         callback=update_flip_coin)

                            
# config callbacks
update_slider_head.args["master_slider"] = head_slider
update_slider_head.args["slave_slider"] = tail_slider
update_flip_coin.args["slider"] = head_slider

In [5]:
# show the final figure
show(column(gridplot([button_one_flip,head_slider],[button_100_flip,tail_slider]),p))

## 1.2 First Concepts

### Terminology

When we later discuss examples that are more complicated than flipping a coin, it will be useful to have an established vocabulary for working with probabilities. A probabilistic **experiment** (such as tossing a coin or rolling a dice) has several components. The **sample space** is the set of all possible **outcomes** in the experiment. We usually denote the sample space by $\Omega$, the Greek capital letter “Omega.” So in a coin toss experiment, the sample space is

$$\Omega = \{H, T\}$$

since there are only two possible outcomes: **heads (H)** or **tails (T)**. Different experiments have different sample spaces. So if we instead consider an experiment in which we roll a standard six-sided dice, the sample space is

$$\Omega = \{1, 2, 3, 4, 5, 6\}$$

Collections of outcomes in the sample space W are called **events**, and we often use capital Roman letters to denote these collections. We might be interested in the event that we roll an even number, for example. If we call this event E, then

$$E = \{2, 4, 6\}$$

Any subset of $\Omega$ is a valid event. In particular, one-element subsets are allowed, so we can speak of the event F of rolling a 4, $F = \{4\}$.


### Assigning probabilities to dice rolls and coin flips

In a random experiment, every event gets assigned a probability. Notationally, if $A$ is some event of interest, then $P(A)$ is the probability that $A$ occurs. The probabilities in an experiment are not arbitrary; they must satisfy a set of **rules** or **axioms**. We first require that **all probabilities be nonnegative**. In other words, in an experiment with
sample space $\Omega$, it must be the case that

$$P(A)\geq 0$$

for any event $A \subset \Omega$. This should make sense given that we’ve already said that a probability of 0 is assigned to an impossible event, and there is no way for something to be less likely than something that is impossible!

The next axiom is that the sum of the probabilities of all the outcomes in $\Omega$ must be 1. We can restate this requirement by the equation

$$\sum_{\omega \in \Omega} P(\omega) = 1$$

Once we know the probabilties of the outcomes in an experiment, we can compute the probability of any event. This is because the probability of an event is the sum of the probabilities of the outcomes it comprises. In other words, for an event $A \subset \Omega$, the probability of A is

$$P(A) = \sum_{\omega \in A} P(\omega)$$

To illustrate this equation, let us find the probability of rolling an even number, an event which we will denote by $E$. Since $E = \{2, 4, 6\}$, we simply add the probabilities of these three outcomes to obtain

$$
\begin{eqnarray*}
\Omega &=& \{1,2,3,4,5,6\}\\
P(E)&=&\sum_{\omega \in E} P(\omega)\\
&=&P(2) + P(4) + P(6)\\
&=&\frac{1}{6} + \frac{1}{6} + \frac{1}{6}\\
&=&\frac{1}{2}
\end{eqnarray*}
$$

Again an example using the coin (...) What is the probability that we get at least one H in two tosses?

$$\Omega = \{HH,HT,TH,TT\}$$

One way to solve this problem is to add up the probabilities of all outcomes that have at least one H, an event which will denote by $E$. Since $E = \{HH,HT,TH\}$ we would get:


$$
\begin{eqnarray*}
P(E) &=& P(HH) + P(HT) + P(TH)\\
&=& p^2 + p  (1 - p) + (1 - p)  p \\
&=& p^2 + 2  (p - p^2)\\
&=& 2  (p - p^2)\\
&=& p  (2 - p)
\end{eqnarray*}
$$



In [6]:
from sympy import *

# enable pretty printing.
init_printing(use_unicode=True)

# create a variable
p = symbols('p')

# simplifying the equation
simplify(p**2 + p * (1-p) + (1-p) * p)

Another way to do this is to find the probability that we don’t flip at least one H, and subtract that probability from 1. This would give us the probability that we do flip at least one H. The only outcome in which we don’t flip at least one H is if we flip T both times. Lets denote this event by F. We would then compute

$$P(F) = P(TT) = (1 - p)^2$$

Then to get the complement of this event, i.e. the event where we do flip at least one H, we subtract the above probability from 1. This gives us

$$
\begin{eqnarray*}
P(E) &=& 1 - P(F)\\
&=& 1 - (1 - p)^2\\
&=& 1 - (1 - 2p + p^2)\\
&=& 2p - p^2\\
&=& p (2 - p)
\end{eqnarray*}
$$


In [7]:
# simplifying the equation
simplify(1 - (1 - p)**2)

**Both methods for solving this problem gave the same answer.** Notice that in the second calculation, we had to sum up fewer probabilities to get the answer. It can often be the case that computing the probability of the complement of an event and subtracting that from 1 to find the probability of the original event requires less work.

## 1.3 Expectation

Consider the outcome of a single dice roll, and call it X. A reasonable question one might ask is “What is the average value of X?". We define this notion of “average” as a weighted sum of outcomes. Since X can take on 6 values, each with probability $\frac{1}{6}$ , the weighted
average of these outcomes should be

$$
\begin{eqnarray*}
\textrm{Weighted Average} &=& \frac{1}{6} \times 1 + \frac{1}{6} \times 2 + \frac{1}{6} \times 3 + \frac{1}{6} \times 4 + \frac{1}{6} \times 5 + \frac{1}{6} \times 6\\
 &=& \frac{1}{6} \times (1 + 2 + 3 + 4 + 5 + 6)\\
 &=& \frac{21}{6}\\
 &=& 3.5
\end{eqnarray*}
$$

This may seem dubious to some. How can the average roll be a noninteger value? The confusion lies in the interpretation of the phrase average roll. A more correct interpretation would be the long term average of the dice rolls. Suppose we rolled the dice many times, and recorded each roll. Then we took the average of all those rolls. This average would be the fraction of 1’s, times 1, plus the fraction of 2’s, times 2, plus the fraction of 3’s, times 3, and so on. But this is exactly the computation we have done above! In the long run, the fraction of
each of these outcomes is nothing but their probability, in this case, 16
for each of the 6 outcomes.

From this very specific dice rolling example, we can abstract the notion of the average value of a random quantity. The concept of average value is an important one in statistics, so much so that it even gets a special bold faced name. Below is the mathematical definition for the
expectation, or average value, of a random quantity X.

**Definition**. The **expected value**, or **expectation** of X, denoted by E(X), is defined to be

$$
E(X) = \sum_{x \in X(\Omega)} x P(X=x)
$$

This expression may look intimidating, but it is actually conveying a very simple set of instructions, the same ones we followed to compute the average value of X.

The $\sum$ sign means to sum over, and the indices of the items we are summing are denoted below the $\sum$ sign. The $\in$ symbol is shorthand for “contained in”, so the expression below the $\sum$ is telling us to sum over all items contained in our sample space $\Omega$. We can think of the expression to the right of the $\sum$ sign as the actual items we are summing, in this case, the weighted contribution of each item in our sample space.


The notation $X(\Omega)$ is used to deal with the fact that $\Omega$ may not be a set of numbers, so a weighted sum of elements in $\Omega$ isn’t even well defined. For instance, in the case of a coin flip, how can we compute $H \times \frac{1}{2} + T \times \frac{1}{2}$? 

We would first need to assign numerical values to H and T in order to compute a meaningful expected value. For a coin flip we typically make the following assignments,

$$
T \longmapsto 0\\
H \longmapsto 1
$$

So when computing an expectation, the indices that we would sum over are contained in the set

$$
X(\Omega) = \{0, 1\}
$$

Let’s use this set of instructions to compute the expected value for a coin flip.

### Expectation of a Coin Flip

Now let X denote the value of a coin flip with bias p. That is, with probability p we flip H, and in this case we say X = 1. Similarly, with probability $1 - p$ we flip T, and in this case we say X = 0. The expected value of the random quantity X is then

$$
\displaystyle \begin{eqnarray*}
E(X) &=& \sum_{x \in X(\Omega)} x P(X=x)\\
 &=& \sum_{x \in \{0,1\}} x P(X=x)\\
 &=& 0 \times P(X = 0) + 1 \times P(X = 1)\\
 &=& 0 \times P(T) + 1 \times P(H)\\
 &=& 0 \times (1 - p) + 1 \times p\\
 &=& p
\end{eqnarray*}
$$

So the expected value of this experiment is p. If we were flipping a fair coin, then $p = \frac{1}{2}$, so the average value of X would be $\frac{1}{2}$.

Again, we can never get an outcome that would yield $X = \frac{1}{2}$, but this is not the interpretation of the expectation of X. Remember, the correct interpretation is to consider what would happen if we flipped the coin many times, obtained a sequence of 0’s and 1’s, and took the average of those values. We would expect around half of the flips to give 0 and the other half to give 1, giving an average value of $\frac{1}{2}$.

In [17]:
from sympy.stats import FiniteRV, E

# unbiased coin
density = {'H': 0, 'T': 1}

# Create a Finite Random Variable given a dict representing the density.
X = FiniteRV('X', density)

print(E(X))

T


In [9]:
# initial expectation
x = range(0,200)
y = [3.5]*200

# all raw data and average results
x_avg = [0]
y_avg = [0]

x_all = [0]
y_all = [0]

# basic data structure 
source = ColumnDataSource(data=dict(x=x, y=y))
source_avg = ColumnDataSource(data=dict(x=x_avg, y=y_avg))
source_all = ColumnDataSource(data=dict(x=x_all, y=y_all))



# create a generic figure
plot = Figure(title="Expectation", 
           plot_height=300, 
           plot_width=600, 
           y_range=[1,6], 
           x_axis_label='Number of rolls',
           y_axis_label='Value')

# add a line to expected value and other to average data
plot_1 = plot.line(x='x', y='y', source=source, line_color="red",line_dash='dashed')
plot.add_tools(HoverTool(renderers=[plot_1], tooltips=[('Expected value', '$y{0.000}')]))

plot_2 = plot.line(x='x', y='y', source=source_avg, line_color="blue")
plot.add_tools(HoverTool(renderers=[plot_2], tooltips=[('Average value', '$y{0.000}')]))


plot.xgrid.grid_line_color = None
plot.ygrid.grid_line_color = None

# callbacks

# After click to a button
# toss the dice and get a value according to probabilities for each face
# Update the plot with average values
update_plot = CustomJS(args=dict(source_all=source_all,
                                 source_avg=source_avg,
                                 source=source), code="""
        data_all = source_all.data
        data_avg = source_avg.data
        data = source.data
        
        x = data['x']
        y = data['y']
        
        x_all = data_all['x']
        y_all = data_all['y']
        
        x_avg = data_avg['x']
        y_avg = data_avg['y']
        
        //probabilities are given by the sliders values
        probs = [slider_1,slider_2,slider_3,slider_4,slider_5,slider_6]
        

        if (cb_obj.label == 'Roll the dice')
            n = 1
        else
            n = 100
            
        for (i =0;  i < n; i++){
            // assume sum of bias is 1
            randRoll = Math.random()
            sum = 0
            result = 1
            for (j = 0; j < 6; j++){
                sum += probs[j].value
                if (randRoll < sum)
                    break
                result+=1
            }
            //add a new results (face)
            x_all.push(x_all[x_all.length - 1]+1)
            y_all.push(result)
            
            //console.log(y_all)
            avg = 0
            //calculate the new average
            for (t = 1; t < y_all.length; t++){
                avg += y_all[t]
            }
            avg = avg/(y_all.length-1)
            
            //add the new average
            x_avg.push(x_avg[x_avg.length - 1]+1)
            y_avg.push(avg)
        }  
        
        //is it need to update the size of source?
        if (y_all.length > x.length){
            diff = y_all.length - x.length
            data['x'] = x.concat(Array.from({length: diff+80}, (v, k) => k+x.length))
            data['y'] = y.concat(Array.from({length: diff+80}, (v, k) => y[1]))
            //console.log(x,y)
        }
        
        source.change.emit()
        source_all.change.emit()
        source_avg.change.emit()
        """)

# select appropriate probabilities for each face of dices
# for each configurations update the plot with new expected values
update_slider = CustomJS(args=dict(source_all=source_all,
                                   source_avg=source_avg,
                                   source=source), code="""
                                   
        data = source.data
        data_all = source_all.data
        data_avg = source_avg.data
        
        //calculate the new probabilites of sliders
        sliders = [slider_1,slider_2,slider_3,slider_4,slider_5,slider_6]
        index = parseInt(cb_obj.title.substring(5, 6)) - 1
        remain = 1 - sliders[index].value
        diff_values = 0
        sum_remain = 0
        for (i = 0; i < 6; i++) {
           if  (i != index)
                sum_remain += sliders[i].value
        }
        
        for (i = 0; i < 6; i++) {
           if (remain != 0)
               diff_values = Math.abs((sum_remain - remain)*(sliders[i].value/remain))
           if  (sum_remain > remain)
                sliders[i].value = Math.max(0, sliders[i].value - diff_values)
           else
                sliders[i].value = Math.min(1, sliders[i].value + diff_values)
        }
        
        //update the plot with the new expectation
        data_all['x'].length = 0
        data_all['y'].length = 0
        
        data_avg['x'].length = 0
        data_avg['y'].length = 0
        
        data['x'].length = 0
        data['y'].length = 0
        
        data_all['x'] = [0]
        data_all['y'] = [0]
        
        data_avg['x'] = [0]
        data_avg['y'] = [0]
        
        //calculate the new expectation
        expectation = 0
        for (i=0; i < 6; i++){
            expectation += sliders[i].value * (i+1)
        }
        
        data['x'] = Array.apply(null, {length: 200}).map(Number.call, Number)
        data['y'] = Array(200).fill(expectation)
        
        //console.log(remain,sum_remain,diff_values)
        
        source.change.emit()
        source_all.change.emit()
        source_avg.change.emit()
        """)

# create widgets
button_roll_the_dice = Button(label="Roll the dice", 
                         button_type="primary",
                         callback=update_plot)

button_roll_times = Button(label="Roll 100 times", 
                         button_type="primary",
                         callback=update_plot)

unbiased = 1/6

slider_1 = Slider(start=0.0, end=1.0, value=unbiased,step=.01, title="Face 1",
                     callback=update_slider,format="0[.]00000")
slider_2 = Slider(start=0.0, end=1.0, value=unbiased,step=.01, title="Face 2",
                     callback=update_slider,format="0[.]00000")
slider_3 = Slider(start=0.0, end=1.0, value=unbiased,step=.01, title="Face 3",
                     callback=update_slider,format="0[.]00000")
slider_4 = Slider(start=0.0, end=1.0, value=unbiased,step=.01, title="Face 4",
                     callback=update_slider,format="0[.]00000")
slider_5 = Slider(start=0.0, end=1.0, value=unbiased,step=.01, title="Face 5",
                     callback=update_slider,format="0[.]00000")
slider_6 = Slider(start=0.0, end=1.0, value=unbiased,step=.01, title="Face 6",
                     callback=update_slider,format="0[.]00000")

# config callbacks
update_slider.args["slider_1"] = slider_1
update_slider.args["slider_2"] = slider_2
update_slider.args["slider_3"] = slider_3
update_slider.args["slider_4"] = slider_4
update_slider.args["slider_5"] = slider_5
update_slider.args["slider_6"] = slider_6

update_plot.args["slider_1"] = slider_1
update_plot.args["slider_2"] = slider_2
update_plot.args["slider_3"] = slider_3
update_plot.args["slider_4"] = slider_4
update_plot.args["slider_5"] = slider_5
update_plot.args["slider_6"] = slider_6


# show the final figure
layout_slide = gridplot([slider_1,slider_2,slider_3,slider_4,slider_5,slider_6],
                       ncols=2)
layout_row = row(button_roll_the_dice,button_roll_times)
show(column(layout_slide,layout_row,plot))

In [10]:
from sympy.stats import FiniteRV, E

density = {1: 0.0780, 2: 0.0780, 3: 0.6100, 4: 0.0780, 5: 0.0780, 6: 0.0780}

# Create a Finite Random Variable given a dict representing the density.
X = FiniteRV('X', density)

print(E(X))

3.23400000000000


<br>
<div class="alert alert-info">
<b>Exercise Start.</b>
</div>

**Show the following properties of expectation using sympy**: 


1. If X and Y are two random variables, then: 
$$E(X + Y) = E(X) + E(Y)$$
2. If X is a random variable and c is a constant, then:
$$E(cX) = cE(X)$$
3. If X and Y are independent random variables, then:
$$E[XY] = E[X]E[Y]$$

In [11]:
from sympy.stats import FiniteRV, E

# unbiased coin
density = {0: 0.5, 1: 0.5}
# Create a Finite Random Variable given a dict representing the density.
X = FiniteRV('X', density)
print('E(X):',E(X))


densityY = {0: 0.4, 1: 0.6}
Y = FiniteRV('Y', densityY)
print('E(Y):',E(Y))

print('E(X+Y):',E(X+Y))


E(X): 0.500000000000000
E(Y): 0.600000000000000
E(X+Y): 1.10000000000000


In [12]:
print('E(cX)',E(3*X))
print('c*E(X)',3*E(X))

E(cX) 1.50000000000000
c*E(X) 1.50000000000000


In [13]:
print('E(X*Y):',E(X*Y))
print('E(X)*E(Y):',E(X)*E(Y))

E(X*Y): 0.300000000000000
E(X)*E(Y): 0.300000000000000


## 1.4 Variance

The variance of a random variable X is a nonnegative number that summarizes on average how much X differs from its mean, or expectation. The first expression that comes to mind is

$$X − E(X)$$

i.e. the difference between X and its mean. This itself is a random variable, since even though EX is just a number, X is still random. Hence we would need to take an expectation to turn this expression into the average amount by which X differs from its expected value. This leads us to

$$E(X − E(X))$$

This is almost the definition for variance. We require that the variance always be nonnegative, so the expression inside the expectation should always be $\geq 0$. Instead of taking the expectation of the difference, we take the expectation of the squared difference.

**Definition**: The variance of X, denoted by Var(X) is defined
$$Var(X) = E[(X − E(X))^2]$$

Below we give some useful properties of the variance.

*Proposition*. If X is a random variable with mean $E(X)$ and $c$ is a real number,

1. $Var(X) ≥ 0$
2. $Var(cX) = c^2Var(X)$
3. $Var(X) = E(X^2) − E(X)^2$

In [14]:
from sympy.stats import variance, Die,E
from sympy import simplify

X = Die('X',6)
print(variance(X))
Y = Die('Y',5)
print(variance(Y))

print('Var(cX) = c^2Var(X):',variance(2*X) == 4*variance(X))
print('Var(X) = E(X^2) − E(X)^2: ',variance(X) == E(X**2)-(E(X)**2))


35/12
2
Var(cX) = c^2Var(X): True
Var(X) = E(X^2) − E(X)^2:  True


<br>
<div class="alert alert-info">
<b>Exercise Start.</b>
</div>

1. Compute the variance of a dice roll, i.e. a uniform random variable over the sample space $\Omega = \{1, 2, 3, 4, 5, 6\}$
$$
\begin{eqnarray*}
VAR(X) &=& E(X^2) - E(X)^2\\
&=& (\sum_{k=1}^{6}k^2 . \frac{1}{6}) - 3.5^2\\
&\approx& 2.92
\end{eqnarray*}
$$
2. Now you can do the same using sympy
3. Calculate the variance assuming that the Dice is biased, which probability density is
```python
    density_ = {1: 0.0780, 2: 0.0780, 3: 0.6100, 4: 0.0780, 5: 0.0780, 6: 0.0780}
```
4. Prove using sympy the propositions
    - $Var(cX) = c^2Var(X)$
    - $Var(X) = E(X^2) − E(X)^2$


In [15]:
from sympy.stats import FiniteRV, E
from sympy.stats import variance, Die
from sympy import simplify

X = Die('X',6)
print('Var(X)=',variance(X))

Var(X)= 35/12


In [16]:
density_ = {1: 0.0780, 2: 0.0780, 3: 0.6100, 4: 0.0780, 5: 0.0780, 6: 0.0780}
X = FiniteRV('X', density)
print('Var(cX) = c^2Var(X):',variance(2*X) == 4*variance(X))
print('Var(X) = E(X^2) − E(X)^2: ',variance(X) == E(X**2)-(E(X)**2))

Var(cX) = c^2Var(X): True
Var(X) = E(X^2) − E(X)^2:  True


# 2 Conditional Probability

Suppose we had a bag that contained two coins. One coin is a fair coin, and the other has a bias of 0.95, that is, if you flip this biased coin, it will come up heads with probability 0.95 and tails with probability 0.05. Holding the bag in one hand, you blindly reach in with
your other, and pick out a coin. You flip this coin 3 times and see that all three times, the coin came up heads. You suspect that this coin is “likely” the biased coin, but how “likely” is it?

This problem highlights a typical situation in which new information changes the likelihood of an event. The original event was “we pick the biased coin”. Before reaching in to grab a coin and then flipping it, we would reason that the probability of this event occurring (picking the biased coin) is $\frac{1}{2}$.

After flipping the coin a couple of times and seeing that it landed heads all three times, we gain new information, and our probability should no longer be $\frac{1}{2}$. In fact, it
should be much higher. In this case, we “condition” on the event of flipping 3 heads out of 3 total flips. We would write this new probability as

$$P(\textrm{picking the biased coin} \  |  \ \textrm{flipping 3 heads out of 3 total flips})$$

The "bar" between the two events in the probability expression above represents "conditioned on", and is defined below.


**Definition** The probability of an event A conditioned on an event B is denoted and defined

$$P(A | B) = \frac{P(A ∩ B)}{P(B)}$$


The intuition of this definition can be gained by playing with the visualization on the website. Suppose we drop a ball uniformly at random in the visualization. If we ask “What is the probability that a ball hits the orange shelf?", we can compute this probability by simply dividing the length of the orange shelf by the length of the entire space. Now suppose we are given the information that our ball landed on the green shelf. What is the probability of landing on the orange shelf now? Our green shelf has become our “new” sample space, and the proportion of the green shelf that overlaps with the orange shelf is now the only region in which we could have possibly landed on the orange shelf. To compute this new conditional probability, we would divide the length of the overlapping, or “intersecting”, regions of the orange and green shelves by the total length of the green shelf.


## 2.1 Bayes Rule

Now that we’ve understood where the definition of conditional probability comes from, we can use it to prove a useful identity.

**Theorem (Bayes Rule)**. Let A and B be two subsets of our sample space $\Omega$. Then

$$P(A | B) = \frac{P(B | A)P(A)}{P(B)}$$


Proof. By the definition of conditional probability,

$$P(A | B) = \frac{P(A ∩ B)}{P(B)}$$

Similarly,

$$P(B | A) = \frac{P(A ∩ B)}{P(A)}$$

Multiplying both sides by P(A) gives

$$P(B | A)P(A) = P(A ∩ B)$$

Plugging this into our first equation, we conclude

$$P(A | B) = \frac{P(B | A)P(A)}{P(B)}$$

### 2.1.1 Coins in a Bag


Let’s return to our first example in this section and try to use our new theorem to find a solution. Define the events

$$A = \{\textrm{Picking the biased coin}\}$$

$$B = \{\textrm{Flipping 3 heads out of 3 total flips} \}$$


We were interested in computing the probability $P(A | B)$. By Bayes Rule,

$$ P(A | B) = \frac{P(B | A)P(A)}{P(B)}$$

$P(B | A)$, i.e. the probability of flipping 3 heads out of 3 total flips given that we picked the biased coin, is simply $(0.95)^3 ≈ 0.857$. The probability $P(A)$, i.e. the probability that we picked the biased coin is $\frac{1}{2}$ since we blindly picked a coin from the bag. Now all we need to do is compute $P(B)$, the overall probability of flipping 3 heads in this
experiment.

$$
\begin{eqnarray*}
P(B) &=& P(B|A)P(A) + P(B|A^c)P(A^c)\\
&=& 0.857 \times 0.5 + 0.125 \times 0.5\\
&=& 0.491
\end{eqnarray*}
$$


Plugging this back into the formula given by Bayes Rule,

$$
\begin{eqnarray*}
P(A | B) &=& \frac{0.857 \times 0.5}{0.491}
&=& 0.873
\end{eqnarray*}
$$
Thus, given that we flipped 3 heads out of a total 3 flips, the probability
that we picked the biased coin is roughly 87.3%.


<br>
<div class="alert alert-info">
<b>Case Study -  Analyzing the Bayes's rule in a real problem </b>
</div>

### Reference

[William Koehrsen's Github](
https://github.com/WillKoehrsen/ai-projects/blob/master/bayesian_inference/bayes_rule_applied.ipynb)
[William's Blog](https://towardsdatascience.com/bayes-rule-applied-75965e4482ff
)    